<a href="https://colab.research.google.com/github/yexf308/MAT592/blob/main/Module3/PyTorch_and_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pylab inline 
import pandas as pd
from scipy import linalg
import scipy

import torch
import torch.nn as nn
import torch.nn.functional as F

Populating the interactive namespace from numpy and matplotlib


# Logistic regression in PyTorch


In [1]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


### Binary Logistic Regression: 9 or not 9


In [18]:
Xtr                   = train_X[0:6000, :]
train_y_part          = train_y[0:6000]
Ytr                   = zeros(train_y_part.size)
Ytr[train_y_part==9]=1

Xte                   = test_X[0:1000,:]
test_y_part           = test_y[0:1000]
Yte  = zeros(test_y_part.size)
Yte[test_y_part==9]=1

Xtr = torch.from_numpy(Xtr)
Xte = torch.from_numpy(Xte)

Xtrain = Xtr.view(-1, 28*28)/256
Xtest  = Xte.view(-1, 28*28)/256

Ytr = torch.from_numpy(Ytr)
Yte = torch.from_numpy(Yte)

In [21]:
#Feel free to play around with these :) 

batch_size = 100
epochs = 10
input_dim = 784
output_dim = 1 #binary classification problem first 
lr_rate = 0.001

#setting up our model - one way to set up models in PyTorch is with nn.Sequential 

binaryMNIST = nn.Sequential(
    nn.Linear(784, 1), 
    nn.Sigmoid())


In [ ]:
#We can look at the untrained weights in the model if we want:
print('\n\nModel params:')
for param in binaryMNIST.parameters():
    print(param)

In [54]:

#set up the loss - binary cross entropy is already implemented by Pytorch
Loss = torch.nn.BCELoss()
#using SGD - we have to tell it what we're going to be optimizing 
optimizer = torch.optim.SGD(binaryMNIST.parameters(), lr=lr_rate)

arr = np.arange(Xtrain.shape[0])
for epoch in range(int(epochs)):
  np.random.shuffle(arr)
  print(epoch)
  for i in arr:
    Xtrain_i = Xtrain[i]
    output_i = binaryMNIST(Xtrain_i)
    loss_i  = Loss(torch.squeeze(output_i), Ytr[0].to(torch.float32))
    optimizer.zero_grad() #make sure to zero gradients out  
    loss_i.backward()
    optimizer.step()



0
1
2
3
4
5
6
7
8
9


In [60]:
correct = 0
for i in range(Xtest.shape[0]):
  Xtest_i = Xtest[i]
  outputs = binaryMNIST(Xtest_i)
  correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == Yte[i].detach().numpy())

accuracy = 100 * correct/Xtest.shape[0]
print("Epoch: {}. Train Loss: {}. Validation Accuracy: {}.".format(epoch, loss_i.item(), accuracy))

Epoch: 9. Train Loss: 8.082716522039846e-05. Validation Accuracy: 90.6.


# MULTICLASS LOGISTIC REGRESSION
